Pretraining: Train the model to just repeat the input sentence.

In [ ]:
START_EPOCH = 0
EPOCHS = 1
CHECKPOINT = "google/mt5-small"
CHECKPOINT_SHORT = "mt5-small"

In [ ]:
# Stuff for running the same notebook locally and on Google Colab for training
import sys

COLAB_PATH = "/content/drive/MyDrive/Colab Notebooks/diversiformer/"
IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    from google.colab import drive  # type: ignore

    drive.mount("/content/drive")
    sys.path.append(COLAB_PATH + "src")
    %pip install transformers datasets sacrebleu sentencepiece carbontracker

In [ ]:
from helpers import read_wiki_sents
from datasets import Dataset

sents = read_wiki_sents(COLAB_PATH if IN_COLAB else None)
data = Dataset.from_dict(dict(x=sents)).train_test_split(0.1)

In [ ]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained(CHECKPOINT)

In [ ]:
def preprocess_function(data):
    inputs = [f"Wiederhole: {d}" for d in data["x"]]
    targets = data["x"]
    model_inputs = tokenizer(inputs)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_data = data.map(preprocess_function, batched=True)

In [ ]:
from transformers import TFT5ForConditionalGeneration

try:
    # load model
    model = TFT5ForConditionalGeneration.from_pretrained(
        (COLAB_PATH if IN_COLAB else "../data/")
        + f"checkpoint_pretrain_{CHECKPOINT_SHORT}_{START_EPOCH}_epochs"
    )
except:
    print("WARNING: Could not load local model.")
    model = TFT5ForConditionalGeneration.from_pretrained(CHECKPOINT)

In [ ]:
from transformers import AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)
model.compile(optimizer=optimizer)

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer, model=model, return_tensors="tf"
)

In [ ]:
tf_train_set = tokenized_data["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = tokenized_data["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [ ]:
examples = """
Wiederhole: Die Botanik definiert Bäume als ausdauernde und verholzende Samenpflanzen, die eine dominierende Sprossachse aufweisen, die durch sekundäres Dickenwachstum an Umfang zunimmt. 
Wiederhole: Diese Merkmale unterscheiden einen Baum von Sträuchern, Farnen, Palmen und anderen verholzenden Pflanzen. 
Wiederhole: Im Gegensatz zu ihren entwicklungsgeschichtlichen Vorläufern verfügen die meisten Bäume zudem über wesentlich differenziertere Blattorgane, die mehrfach verzweigten Seitentrieben (Lang- und Kurztrieben) entspringen. 
Wiederhole: Stamm, Äste und Zweige verlängern sich jedes Jahr durch Austreiben von End- und Seitenknospen, verholzen dabei und nehmen kontinuierlich an Umfang zu. 
Wiederhole: Im Gegensatz zum Strauch ist es besonderes Merkmal der Bäume, dass die Endknospen über die Seitenknospen dominieren (Apikaldominanz) und sich dadurch ein vorherrschender Haupttrieb herausbildet (Akrotonie). 
""".strip().split(
    "\n"
)

In [ ]:
def generate(prompt, model, tokenizer):
    tokenized_text = tokenizer.encode(prompt, return_tensors="tf")
    summary_ids = model.generate(tokenized_text, max_length=tokenized_text.shape[1])
    output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return output

In [ ]:
if IN_COLAB:
    import json
    from carbontracker.tracker import CarbonTracker

    tracker = CarbonTracker(epochs=EPOCHS, verbose=2)
    try:
        with open(
            COLAB_PATH + f"example_predictions_pretrain_{CHECKPOINT_SHORT}.json"
        ) as f:
            example_eval = json.load(f)
    except:
        example_eval = []
    example_eval = []
    for epoch in range(1, EPOCHS + 1):
        print(f"Epoch {START_EPOCH + epoch}")
        tracker.epoch_start()
        model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=1)
        tracker.epoch_end()
        example_eval.append(
            [
                dict(
                    epoch=START_EPOCH + epoch,
                    prompt=prompt,
                    response=generate(prompt, model, tokenizer),
                )
                for prompt in examples
            ]
        )
        with open(
            COLAB_PATH + f"example_predictions_pretrain_{CHECKPOINT_SHORT}.json", "w"
        ) as f:
            json.dump(example_eval, f, indent=2, ensure_ascii=False)
        if epoch % 1 == 0:
            model.save_pretrained(
                COLAB_PATH
                + f"checkpoint_pretrain_{CHECKPOINT_SHORT}_{START_EPOCH+epoch}_epochs"
            )
            model.save(
                COLAB_PATH
                + f"tf_checkpoint_pretrain_{CHECKPOINT_SHORT}_{START_EPOCH+epoch}_epochs"
            )
    tracker.stop()

In [ ]:
for s in sents[:20]:
    prompt = f"Wiederhole: {s}"
    output = generate(prompt, model, tokenizer)
    print(prompt[12:] == output)
    if not (prompt[12:] == output):
        print(prompt[12:])
        print(output)
        print()